# Welcome to Knuckledragger!

This tutorial can be used online at 
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/philzook58/knuckledragger/blob/main/tutorial.ipynb)

In [ ]:
%%bash
git clone https://github.com/philzook58/knuckledragger.git
cd knuckledragger
python3 -m pip install -e .

In [ ]:
! cd knuckledragger && ./install.sh # get and build external solvers

# Intro to Z3

Knuckledragger is heavily based around the prexisting SMT solver Z3. Knowing about Z3 is useful even if you aren't interested 

You can find a tutorial notebook and video I made here https://github.com/philzook58/z3_tutorial


In [2]:
from z3 import *

We then declare our variables, state a set of constraints we wish to hold, and then call the convenience function solve to get a solution.

In [3]:
x = Int('x')
y = Int('y')
solve(x > 2, y < 10, x + 2*y == 7)

[y = 0, x = 7]


We can also use z3 to prove properties and theorems.

In [4]:
p = Bool("p")
my_true_thm = Implies(p, p)
my_true_thm

Implies(p, p)

In [5]:
prove(my_true_thm)

proved


If the property is not true, smt can supply a counterexample. 

In [6]:
q = Bool("q")
my_false_thm = Implies(q, p)
my_false_thm

Implies(q, p)

In [7]:
prove(my_false_thm)

counterexample
[p = False, q = True]


# Why Knuckledragger?

Z3 is a tour de force. It's beautiful. However, it falls short of solving or verifying many kinds of problems we may be interested in.

1. It may in principle be able to solve something, but times out
2. Many mathematical and verification questions involve induction, which Z3 does not have strong built in support for. In general automating this is a tough problem
3. We may want to spell out what Z3 can do for use automatically, to make sure we understand
4. Z3 may not have good built in understanding of problem domains other systems like Mathematica and sympy do
5. A systematic approach to hide or abstract things z3 should not care about

For all these reasons, we may want a principled way to use Z3 to confirm 

In comes Knuckledragger.

In principle, Knuckledragger is designed to be extremely minimalist. It can be implemented as a small number of idioms. There is a library however.

1. Lots of little niceties. Pretty printing, useful combinators, tactics. The more easily you can write your statements, the more likely they are to correspond to what's in your head
2. A library of axiomatized theories
3. A bit of rigidity to guide you towards proofs that are more likely to be sound. 


In [12]:
def axiom(thm):
    return thm

def lemma(thm,by=[]):
    prove(Implies(And(by), thm))
    return thm

1. Distinguishing between theorems to be proven and theorems that have been proven
2. It is convenient and builds confidence to record a trace of the result coming in through these functions. These traces are recorded in a `Proof` tree which exactly reflects the call tree to `lemma` and `axiom` that produced to the Proof.

In [11]:
from dataclasses import dataclass

@dataclass
class Proof:
    thm: z3.BoolRef
    reasons: list["Proof"]

def axiom(thm):
    return Proof(thm,[])

def lemma(thm,by=[]):
    assert all(isinstance(x,Proof) for x in by)
    prove(Implies(And(by), thm))
    return Proof(thm,by)

This is in essence the contents of `knuckledragger.kernel`.

# Knuckledragger

But this has been packaged up for you in the `knuckledragger` package alongside many other goodies.

In [3]:
import kdrag as kd
import kdrag.smt as smt # z3 is re-exported as kdrag.smt
# or from kdrag.all import *

In [ ]:
G = smt.DeclareSort("G")
inv = smt.Function("inv", G, G)
mul = smt.Function("mul", G, G, G)
kd.notation.mul.register(G, mul) # enables e * inv(e) notation
e = smt.Const("e", G)
x,y,z = smt.Consts("x y z", G)
mul_assoc = kd.axiom(smt.ForAll([x,y,z], x * (y * z) == (x * y) * z))
mul_id = kd.axiom(smt.ForAll([x], x * e == x))
mul_inv = kd.axiom(smt.ForAll([x], x * inv(x) == e))

group_db = [mul_assoc, mul_id, mul_inv]
def glemma(thm,by=[]):
    return kd.lemma(thm,by + group_db)



